## Hello Delta Lake 

In [4]:
# PreRequisites
import pyspark
from delta import *

In [5]:

builder = pyspark.sql.SparkSession.builder.appName("LocalDelta") \
    .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension") \
    .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog")

spark = configure_spark_with_delta_pip(builder).getOrCreate()

In [ ]:
spark.sparkContext.getConf().getAll()

In [6]:
data = spark.range(0, 5)
data.write.format("delta").save("/tmp/delta-table2")

In [7]:
# Read data from the Delta table
df = spark.read.format("delta").load("/tmp/delta-table2")
df.show()

+---+
| id|
+---+
|  2|
|  1|
|  3|
|  0|
|  4|
+---+



---

### Load data into Delta Lake Table

In [14]:
file_path = "data/loans/loan-risks.snappy.parquet"

# Configure Delta Lake path
delta_path = "/tmp/loans_delta"

# Create the Delta Lake Table
(spark.read.format("parquet").load(file_path).write.format("delta").save(delta_path))

# Create a View on the data 
spark.read.format("delta").load(delta_path).createOrReplaceTempView("loans_delta_tbl")

Py4JJavaError: An error occurred while calling o76.save.
: org.apache.spark.SparkClassNotFoundException: [DATA_SOURCE_NOT_FOUND] Failed to find the data source: delta. Please find packages at `https://spark.apache.org/third-party-projects.html`.
	at org.apache.spark.sql.errors.QueryExecutionErrors$.dataSourceNotFoundError(QueryExecutionErrors.scala:724)
	at org.apache.spark.sql.execution.datasources.DataSource$.lookupDataSource(DataSource.scala:647)
	at org.apache.spark.sql.execution.datasources.DataSource$.lookupDataSourceV2(DataSource.scala:697)
	at org.apache.spark.sql.DataFrameWriter.lookupV2Provider(DataFrameWriter.scala:863)
	at org.apache.spark.sql.DataFrameWriter.saveInternal(DataFrameWriter.scala:257)
	at org.apache.spark.sql.DataFrameWriter.save(DataFrameWriter.scala:240)
	at java.base/jdk.internal.reflect.NativeMethodAccessorImpl.invoke0(Native Method)
	at java.base/jdk.internal.reflect.NativeMethodAccessorImpl.invoke(NativeMethodAccessorImpl.java:77)
	at java.base/jdk.inter